In [1]:
from pyspark.sql import SparkSession
import os

In [2]:
minio_endpoint = os.getenv("STORAGE_URI", "http://222.255.214.74:9000")
minio_access_key = os.getenv("MINIO_ACCESS_KEY", "admin")
minio_secret_key = os.getenv("MINIO_SECRET_KEY", "password")
MINIO_S3_FILE_PATH = "s3a://csv-data-files/HR_Data_MNC_Data.csv"

spark = (SparkSession.builder.appName("PostgreSQL ingestion from file")
         .config("spark.jars.packages",
                 ",".join(["org.postgresql:postgresql:42.7.4",
                           "org.apache.hadoop:hadoop-aws:3.3.4",
                           "com.amazonaws:aws-java-sdk-bundle:1.12.262"
                           ]))
         # Cap driver and executor heaps:
         .config("spark.driver.memory", "2g")
         .config("spark.executor.memory", "2g")
         .getOrCreate())
jdbc_url = os.getenv("JDBC_URL", "jdbc:postgresql://localhost:5435/mydb")
file_path = "../../csv_files/HR_Data_MNC_Data.csv"

target_table = "public.hr_data"
db_user = "myuser"
db_password = "mypassword"

use_minio = True

if use_minio:
    # Configure S3A to talk to MinIO
    hconf = spark.sparkContext._jsc.hadoopConfiguration()
    hconf.set("fs.s3a.endpoint", minio_endpoint)  # MinIO endpoint
    hconf.set("fs.s3a.access.key", minio_access_key)
    hconf.set("fs.s3a.secret.key", minio_secret_key)
    hconf.set("fs.s3a.path.style.access", "true")  # required for MinIO
    hconf.set("fs.s3a.connection.ssl.enabled", "false")  # set true if your MinIO uses HTTPS
    hconf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")



25/08/30 08:46:59 WARN Utils: Your hostname, MacBook-Air-cua-Ngoc-2.local resolves to a loopback address: 127.0.0.1; using 192.168.1.5 instead (on interface en0)
25/08/30 08:46:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/truongngocson/.ivy2/cache
The jars for the packages stored in: /Users/truongngocson/.ivy2/jars
org.postgresql#postgresql added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-87e7b32f-e3e8-408a-8f56-6537bba83321;1.0
	confs: [default]
	found org.postgresql#postgresql;42.7.4 in central


:: loading settings :: url = jar:file:/Users/truongngocson/Documents/Projects/apache-iceberg/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.checkerframework#checker-qual;3.42.0 in local-m2-cache
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 185ms :: artifacts dl 12ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.checkerframework#checker-qual;3.42.0 from local-m2-cache in [default]
	org.postgresql#postgresql;42.7.4 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   5   |   0   |   0   |   0

Read data into dataframe

In [3]:
if use_minio:
    print("Start reading file from MinIO")
    df = (
        spark.read
        .option("header", "true")
        .option("inferSchema", "true")  # consider providing an explicit schema for production
        .csv(MINIO_S3_FILE_PATH)
    )
else:
    df = spark.read.csv(file_path, header=True, inferSchema=True)

df.describe()
df.head()

Start reading file from MinIO


25/08/30 08:47:02 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


Row(Unnamed: 0=0, Employee_ID='EMP0000001', Full_Name='Joshua Nguyen', Department='IT', Job_Title='Software Engineer', Hire_Date=datetime.date(2011, 8, 10), Location='Isaacland, Denmark', Performance_Rating=5, Experience_Years=14, Status='Resigned', Work_Mode='On-site', Salary_INR=1585363)

Start ingesting data

In [4]:
print(f'Starting ingestion into PostgreSQL with connection: {jdbc_url}')

write = (df.write.format("jdbc")
         .option("url", jdbc_url)
         .option("dbtable", target_table)
         .option("user", db_user)
         .option("password", db_password)
         .option("driver", "org.postgresql.Driver")
         #.option("batchsize", 10000)
         .option("truncate", "true")
         .option("createTableOptions", "WITH (OIDS=FALSE)")
         # Create table options (used if the table doesn't exist and Spark creates it):
         )

write.mode("append").save()
print("Done!")

Starting ingestion into PostgreSQL with connection: jdbc:postgresql://222.255.214.74:5435/mydb


Done!


Shutdown session

In [5]:
spark.stop()